In [1]:
include("temporal_mem_thresholder.jl")

Test Summary: | Pass  Total
update!       |    7      7
  94.454 ms (4916 allocations: 6.77 MiB)
nothing


In [ ]:
function update!(tmt::TMThreshold, segment_activations, active_col_idxs)
    
    # Make active column array
    x = active_cols_to_sparray(tmt, active_col_idxs)
    # Make unpredicted set for false negative updates
    unpredicted = Set(active_col_idxs)
    
    # Iterate over synapse activations
    # Update true positives and false positives
    map(enumerate(segment_activations)) do (j, sj)
        θj = @view tmt.θ[:, j]
        aj = SparseVector(sj .> θj)
        update_true_positive!(j, aj, x, tmt, active_col_idxs, unpredicted)
        update_false_pos!(j, aj, x, tmt)
    end
    
    # False negative update
    decrease_threshold!.(tmt, find_max_segments(tmt, segment_activations, unpredicted))
    very_small_increase_threshold!(tmt)
    return nothing
end

In [22]:
num_cols = 1024
cells_per_col = 32
num_cells = num_cols * cells_per_col

# Segments and synapses
segments_per_cell = 64
synapses_per_segment = 50
segment_sparsity = synapses_per_segment / num_cells;

# Initialize
tmt = TMThreshold(
    num_cells, 
    cells_per_col, 
    segments_per_cell, 
    synapses_per_segment
)

enc_len = 32
active_col_idxs = randperm(num_cols)[1:enc_len]
x = active_cols_to_sparray(tmt, active_cols)
segment_activations = project(tmt, x);

In [23]:
# Make active column array
@btime x = active_cols_to_sparray($tmt, $active_col_idxs);


  30.504 μs (188 allocations: 465.30 KiB)


In [24]:
# Make unpredicted set for false negative updates
@btime unpredicted = Set($active_col_idxs);

  462.660 ns (7 allocations: 1.15 KiB)


In [25]:
# Iterate over synapse activations
# Update true positives and false positives
@btime map(enumerate($segment_activations)) do (j, sj)
    θj = @view $tmt.θ[:, j]
    aj = SparseVector(sj .> θj)
    update_true_positive!(j, aj, $x, $tmt, $active_col_idxs, $unpredicted)
    update_false_pos!(j, aj, $x, $tmt)
end;

  87.948 ms (1025 allocations: 5.41 MiB)


# In main loop

In [26]:
@btime θj = @view $tmt.θ[:, 1];

  2.086 ns (0 allocations: 0 bytes)


In [27]:
θj = 4*ones(size(tmt.θ)...)[:, 1]
sj = segment_activations[1]
aj = @btime aj = SparseVector($sj .> $θj);

  411.455 μs (16 allocations: 1.31 MiB)


In [28]:
j = 1
@btime update_true_positive!($j, $aj, $x, $tmt, $active_col_idxs, $unpredicted)


  1.485 μs (6 allocations: 9.62 KiB)


In [29]:
@btime update_false_pos!($j, $aj, $x, $tmt)

  3.503 μs (4 allocations: 33.50 KiB)


### Thresholding seems like the bottleneck

In [32]:
@btime map(enumerate($segment_activations)) do (j, sj)
    θj = @view $tmt.θ[:, j]
    aj = SparseVector(sj .> θj)
end;

  87.709 ms (385 allocations: 2.79 MiB)


This was the problem! Fixed! 5ms per update now

# Outside main loop

In [10]:
# False negative update
@btime decrease_threshold!.(tmt, find_max_segments(tmt, segment_activations, unpredicted));


  55.241 μs (150 allocations: 33.72 KiB)


1-element Vector{Float64}:
 -7816.800000010906

In [13]:
@btime very_small_increase_threshold!(tmt);

  1.124 ms (0 allocations: 0 bytes)
